# Generic Box
This code communicates with a component. The component was launched with the command below from the root of the repository:

```shell
$ docker run --rm -it -p 4061:8061 -v ./src/external1.py=/workspace/external.py generic
```


In [9]:
import argparse
import grpc
import io
import matplotlib.pyplot as plt
#In case of importing files from other directory
from importlib.machinery import SourceFileLoader
simplebox_pb2 = SourceFileLoader("simplebox_pb2","../src/simplebox_pb2.py").load_module()
simplebox_pb2_grpc = SourceFileLoader("simplebox_pb2_grpc","../src/simplebox_pb2_grpc.py").load_module()
#import simplebox_pb2
#import simplebox_pb2_grpc
import PIL.Image as PIL_image
import numpy as np
import cv2
from scipy.io import loadmat,savemat

Auxiliary functions - dipslay image

In [10]:

def display_image(image):
    if isinstance(image,np.ndarray):
        img =image
    else:
        img = PIL_image.open(io.BytesIO(image.data))
    ax = plt.gca()
    ax.imshow(img)
    plt.show()


## Set parameters (GRPC) and data
Set IP and port of the service (the same used in docker run), input data (image in a matfile) and call the service

In [15]:
#target='printart.isr.ist.utl.pt:8061'
target='localhost:8061'
# List of file paths (example paths — replace with your actual ones)
file_paths = [
    'images/00.jpg',
    'images/01.jpg'
]

# List to hold binary data
imgdata = []

# Read binary contents of each file and append to imgdata
for path in file_paths:
    with open(path, 'rb') as f:
        data = f.read()
        imgdata.append(bytearray(data))  # Use bytearray for .mat compatibility

# Save the imgdata list to a .mat file
savemat('imgdata.mat', {'imgdata': imgdata})

print("Saved imgdata to imgdata.mat")

data_file='imgdata.mat'
#Para imagens muito grandes buffer grande
channel_opt = [('grpc.max_send_message_length', 512 * 1024 * 1024), ('grpc.max_receive_message_length', 512 * 1024 * 1024)]
channel=grpc.insecure_channel(target,options=channel_opt)
estimator_stub = simplebox_pb2_grpc.SimpleBoxServiceStub(channel)

Saved imgdata to imgdata.mat


#### Do your task. Example: read an image from a matlab file and send it to process

In [16]:

with open(data_file, 'rb') as fp:
        data_bytes = fp.read()
request = simplebox_pb2.matfile(data=data_bytes)
#Name of the method in the service
response=estimator_stub.process(request)

#After finishing channel.close

### Run in one single shot 

In [19]:
target='printart.isr.ist.utl.pt:8061'
data_file='../Images/eiffel.mat'
#Para imagens muito grandes buffer grande
channel_opt = [('grpc.max_send_message_length', 512 * 1024 * 1024), ('grpc.max_receive_message_length', 512 * 1024 * 1024)]

with grpc.insecure_channel(target,options=channel_opt) as channel:
    estimator_stub = simplebox_pb2_grpc.SimpleBoxServiceStub(channel)
    with open(data_path, 'rb') as fp:
        data_bytes = fp.read()
    request = simplebox_pb2.matfile(data=data_bytes)
    response=estimator_stub.process(request)
    with open("lixo.mat","wb") as f:
    f.write(response.data)
#channel.close()

# Check output

In [34]:
with open("lixo.mat","wb") as f:
    f.write(response.data)

In [35]:
a=loadmat("lixo.mat")
a.keys()

dict_keys(['__header__', '__version__', '__globals__', 'pose_enc', 'depth', 'depth_conf', 'world_points', 'world_points_conf', 'images'])